## Setup...

In [ ]:
# Various setup, probably mostly unused

from env import TestEnv
import pandas as pd
import json
import os
from conf import LisaLogging
from bart.common.Utils import area_under_curve
from trappy.plotter import plot_trace
from IPython.display import display
from trappy import ILinePlot
from trappy.stats.grammar import Parser
from bart.sched.SchedMultiAssert import SchedMultiAssert
import pandas as pd
from trace import Trace
LisaLogging.setup()
import logging
logging.getLogger('Trace').setLevel(logging.ERROR)
logging.getLogger('Analysis').setLevel(logging.WARNING)
logging.getLogger('EnergyMeter').setLevel(logging.DEBUG)
%matplotlib inline
from platforms.juno_energy import juno_energy
from platforms.pixel_energy import pixel_energy
import tests.eas.generic
from tests.eas.generic import EnergyModelTest
import numpy as np
from IPython.display import display
from wlgen import RTA, Periodic
from executor import Executor

In [ ]:
# Some nonsense to get caiman to work on Brendan's computer
p = os.getenv('PATH').split(':')
caiman_path = '/opt/ds5_v5.23.0/bin'
if caiman_path not in p:
    p.insert(0, caiman_path)
    os.environ['PATH'] = ':'.join(p)

In [4]:
from platforms.hikey_energy import hikey_energy

In [ ]:
te = TestEnv(test_conf={
        'modules': ['cgroups'], 
        'ftrace': {
            'events': ['sched_switch', 'cpu_frequency', 'cpu_idle', 'sched_load_avg_cpu', 'sched_load_avg_task', 'irq*']
        }
    }, force_new=True)

In [ ]:
# Some nonsense to make LISA treat HiKey with the respect it deserves
from trappy.stats.Topology import Topology
hikey_topology = Topology(clusters=[[0, 1, 2, 3], [4, 5, 6, 7]])
te.topology = hikey_topology
te.platform['clusters'] = {'big': [0, 1, 2, 3], 'little': [4, 5, 6, 7]}

In [ ]:
def spread_evenly(ntasks, ncpus):
    ret = [0 for _ in range(ncpus)]
    for i in range(ntasks):
        ret[i % ncpus] += 1
    return ret

def get_various_packing_placements(ntasks, ncpus):
    max_spread = spread_evenly(ntasks, ncpus)
    
    max_pack = [ntasks] + [0] * (ncpus - 1)
    
    half_pack = [ntasks / 2] + spread_evenly(ntasks - (ntasks / 2), ncpus -1)
    
    return [max_spread, half_pack, max_pack]

In [ ]:
def get_everything(nrg_model, cpu=0, task_pct=4):
    cpu_node = nrg_model.cpu_nodes[cpu]
    cluster_node = cpu_node.parent
    ncpus = len(cluster_node.cpus)
    min_cap = cpu_node.min_capacity
    task_cap = int(1024 * (task_pct / 100.))
    
    tasks_per_cpu = int(min_cap / task_cap)
    ntasks = tasks_per_cpu * ncpus
    task_distribs = get_various_packing_placements(ntasks, ncpus)
    
    util_distribs = []
    wloads = []
    for task_distrib in task_distribs:
        util_distrib = [0 for _ in range(len(nrg_model.cpus))]
        for i, num_tasks_on_cpu in enumerate(task_distrib):
            util_distrib[cluster_node.cpus[i]] = num_tasks_on_cpu * task_cap
        util_distribs.append(util_distrib)

    # Stagger the workloads so that they prevent shared idle states
    period_s = 10e-3
    stagger_s = 0.7e-3
    delays = np.arange(period_s, step=stagger_s)
    
    if len(delays) > ntasks:
        print "WARNING: not enough tasks to cover period with wakeups"
    
    tasks = {}
    for i in range(ntasks):
        tasks['{}pct_{}'.format(task_pct, i)] = {
            'kind': 'Periodic',
            'params': {
                'duty_cycle_pct': task_pct,
                'period_ms' : period_s * 1e3 + (i - (ntasks / 2)),
                'delay_s' : delays[i % len(delays)],
                'duration_s': 2
            }
        }
    
    wloads = {
        'forced_pack': {
            'type': 'rt-app',
            'conf': {
                'class': 'profile',
                'params': tasks,
                'cpus': [0, 1, 2, 3]
            }
        },
        'any_cpu': {
            'type': 'rt-app',
            'conf': {
                'class': 'profile',
                'params': tasks,
            }
        }
    }
        
    return task_distribs, util_distribs, wloads

In [ ]:
task_distribs, util_distribs, wloads = get_everything(hikey_energy)

In [ ]:
poor_mans_powersave = { # No powersave governor in hikey_defconfi
    'governor': 'userspace',
    'freqs':{
        0: 208000,
    }
}
performance = {
    'governor': 'performance'
}
executor = Executor(te, {
        'confs': [{
                'tag': 'myconf',
                'flags': ['ftrace', 'freeze_userspace'],
                'cpufreq': performance,
            }],
        'wloads': wloads,
        'iterations': 5
    })

In [ ]:
executor.run()

In [ ]:
traces = [Trace(te.platform, e.out_dir, ['sched_switch', 'cpu_idle', 'cpu_frequency']) for e in executor.experiments]

In [ ]:
for trace in traces:
    plot_trace(trace.ftrace)

In [ ]:
clusters = [[0, 1, 2, 3], [4, 5, 6, 7]]

In [ ]:
for trace in traces:
    ca_signals = []
    for cluster in clusters:
        ca_signals.append(pd.DataFrame(trace.getClusterActiveSignal(cluster)))
    ILinePlot(ca_signals, column=0, drawstyle='steps-post', fill=True, fill_alpha=0.8).view() 
    trace.analysis.idle.plotClusterIdleStateResidency()

In [ ]:
def examine_experiment(experiment):
    trace = Trace(te.platform, experiment.out_dir, ['sched_switch'])
    ma = SchedMultiAssert(trace.ftrace, hikey_topology, experiment.wload.tasks.keys())
    cluster_conclusions = []
    for cluster in te.topology.get_level('cluster'):
        residencies = [t['residency'] for p, t in ma.getResidency('cluster', cluster, percent=True).iteritems()]
        if all(r > 90 for r in residencies):
            cluster_conclusions.append('PACKED')
        else:
            cluster_conclusions.append(None)
    packed = [i for i, c in enumerate(cluster_conclusions) if c == 'PACKED']
    with open(os.path.join(experiment.out_dir, 'energy.json')) as f:
        energy = json.load(f)
        energy = energy['BAT']
    if not packed:
        print "I don't think we packed onto any cluster"
        return False, energy
        # plot_trace(trace.ftrace)
    elif len(packed) == 1:
        [i] = packed
        print 'I think we packed onto cluster {} ({})'.format(i, te.topology.get_node('cluster', i))
        return True, energy
    else:
        # this code is
        print 'what is this'
        # who even.. who wrote this shit
        print 'i dont even'
        # Microsoft recommends Windows 10 with Microsoft Edge® for a more secure browsing experience
        raise Exception(RuntimeError(GenericBamboozlementComplaintFactory2().generateComplaint('bamboozled!!!')))

In [ ]:
previous_energy_records = [[], []]

In [ ]:
energy_records = previous_energy_records
for experiment in executor.experiments:
    packed, energy = examine_experiment(experiment)
    energy_records[packed].append(energy)

In [ ]:
import numpy as np

In [ ]:
np.mean(energy_records[0]), np.std(energy_records[0])

In [ ]:
np.mean(energy_records[1]), np.std(energy_records[1])

In [ ]:
energy_records

In [ ]:
traces = [Trace(te.platform, e.out_dir, ['sched_switch', 'cpu_frequency']) for e in executor.experiments]

In [ ]:
te.platform

In [ ]:
freqs = [208000, 432000, 729000, 960000, 1200000]
te.platform['freqs']['big'] = freqs
te.platform['freqs']['little'] = freqs

In [ ]:
trace = traces[0]
trace.analysis.frequency.plotClusterFrequencies()

In [ ]:
te.target.big_core

In [ ]:
trace.analysis.tasks.plotTasks()

.

..

..

..

..

..

..

..

..

..

..

..

..

..

..

..

..

..

..

..

..

..

.

In [ ]:
raise Exception('HALT HALT HALT, HALT AT THE GATES OF THE GRAVEYARD OR SURELY PERISH')

##### 💀💀 The code graveyard 💀💀
*abandon hope all ye who scroll past here, for here is the threshold of the graveyard, whither is sent the code of yore, remembered but not missed, respected but not loved, written by the hand of fearful men and trialled by the silicon of scorned machines*

In [ ]:
def get_task_distribs(nrg_model, cpu=0, task_pct=4):
    cpu_node = nrg_model.cpu_nodes[cpu]
    cluster_node = cpu_node.parent
    ncpus = len(cluster_node.cpus)
    min_cap = cpu_node.min_capacity
    
    task_cap = int(1024 * (task_pct / 100.))
    
    tasks_per_cpu = int(min_cap / task_cap)
    task_distribs = get_various_packing_placements(tasks_per_cpu * ncpus, ncpus)
    return task_distribs

In [ ]:
def get_util_distribs(nrg_model, task_distribs=None, cpu=0, task_pct=4):
    if task_distribs == None:
        task_distribs = get_task_distribs(nrg_model, cpu, task_pct)
    util_distribs = []
    
    task_cap = int(1024 * (task_pct / 100.))
    cpu_node = nrg_model.cpu_nodes[cpu]
    cluster_node = cpu_node.parent
    
    for task_distrib in task_distribs:
        util_distrib = [0 for _ in range(len(nrg_model.cpus))]
        for i, num_tasks_on_cpu in enumerate(task_distrib):
            util_distrib[cluster_node.cpus[i]] = num_tasks_on_cpu * task_cap
        util_distribs.append(util_distrib)
    return util_distribs

In [ ]:
def get_wloads(nrg_model, task_distribs=None, cpu=0, task_pct=4):
    if task_distribs == None:
        task_distribs = get_task_distribs(nrg_model, cpu, task_pct)
    for task_distrib in 

In [ ]:
util_distribs = get_util_distribs(hikey_energy)
for ud in util_distribs:
    display(hikey_energy.estimate_from_cpu_util(ud))

In [ ]:
hikey_energy.cpu_nodes[0].min_capacity

In [ ]:
def get_cluster_count_df(experiment, clusters, trace=None):
    if trace is None:
        trace = Trace(te.platform, experiment.out_dir, ['sched_switch'])
    tasks = experiment.wload.tasks.keys()
    pids = [trace.getTaskByName(t)[0] for t in tasks]
    task_cpus = trace.data_frame.task_cpus()[pids]
    # drop consecutive duplicates
    task_cpus = task_cpus[(task_cpus.shift(+1) != task_cpus).any(axis=1)]

    df = pd.DataFrame(columns=[str(c) for c in clusters])
    
    for (time, row_in) in task_cpus.iterrows():
        cluster_task_counts = [0 for c in clusters]
        for task_cpu in row_in:
            if isnan(task_cpu):
                continue
            for i, cluster in enumerate(clusters):
                if int(task_cpu) in cluster:
                    cluster_task_counts[i] += 1
                    break
    
        df.loc[time] = pd.Series(cluster_task_counts, index=df.columns)
        
    return df

def were_tasks_cluster_packed(experiment, clusters, trace=None):
    if trace is None:
        trace = Trace(te.platform, experiment.out_dir, ['sched_switch'])
    cluster_counts = get_cluster_count_df(experiment, clusters, trace)
    

In [ ]:
def get_nrg(model, util):
    nrg = model.estimate_from_cpu_util(util, zero_idle=True)
    return sum(nrg.values())